# 전처리 과정에 필요한 패키지 불러오기 및 명령어 설정

In [1]:
import nltk

In [2]:
import os
os.getcwd()

'/Users/seohyejin/Study/2019_deep_learning_preprocessing/200118_Step1_BERT'

In [3]:
import sys
sys.path
sys.path.append('/Users/seohyejin/Study/frequently_used') # contractions_sentiment.py 파일이 저장되어있는 디렉토리
print(sys.path)

['/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python36.zip', '/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6', '/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload', '', '/usr/local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages/IPython/extensions', '/Users/seohyejin/.ipython', '/Users/seohyejin/Study/frequently_used']


In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords 
from contractions_sentiment import CONTRACTION_MAP
from nltk.stem import WordNetLemmatizer

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

stemmer = nltk.stem.SnowballStemmer('english')
lemma=WordNetLemmatizer()

from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

lemmatizer = WordNetLemmatizer() 

In [5]:
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [6]:
lemmatizer = WordNetLemmatizer() 
  
print("rocks :", lemma.lemmatize("rocks")) 
print("corpora :", lemma.lemmatize("corpora")) 
print("was :", lemma.lemmatize("was")) 
print("were :", lemma.lemmatize("were")) 
print("is :", lemma.lemmatize("is")) 

rocks : rock
corpora : corpus
was : wa
were : were
is : is


In [7]:
print(lemma.lemmatize("was", 'v'))  

be


In [8]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions("Y'all can't expand contractions I'd think")

'You all can not expand contractions I would think'

In [9]:
expand_contractions("isnt")

'is not'

In [10]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3', 'lol', 'LOL'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])


#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

# 트위터 데이터 불러오기

In [11]:
def ingest():
    data = pd.read_csv('/Users/seohyejin/Study/2019_deep_learning_preprocessing/trainingandtestdata/tweets.csv', encoding='latin-1') # 파일 저장소 디렉토리
    data.columns=["Sentiment","ItemID","Date","Blank","SentimentSource","SentimentText"]
    data.drop(['ItemID', 'SentimentSource'], axis=1, inplace=True)
    data = data[data.Sentiment.isnull() == False]
    data['Sentiment'] = data['Sentiment'].map( {4:1, 0:0}) #Converting 4 to 1
    data = data[data['SentimentText'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape  )  
    return data

data = ingest()

dataset loaded with shape (1599999, 4)


In [12]:
data.iloc[:3]

,Sentiment,Date,Blank,SentimentText
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire


In [13]:
# 원본 데이터가 160만 개에서 한 개가 부족해서 데이터 한 개 추가

In [14]:
df = pd.DataFrame(columns=['Sentiment', 'Date', 'Blank', 'SentimentText'])

In [15]:
df.loc[0] = [0,'Mon Apr 06 22:20:20 PDT 2009','NO_QUERY', "@smarrison i would've been the first, but i didn't have a gun. not really though, zac snyder's just a doucheclown."]

In [16]:
df2 = data.append(df , ignore_index=True)

In [17]:
len(df2)

1600000

In [18]:
data = df2

In [19]:
def data_text_cleaning_lemma(data):
    
    tweet = data.lower().strip() # 소문자화
    tweet = tweet.replace("’","'") # 작은 따움표 표기법 바꾸기 
    #print(tweet)
    
    if tweet.startswith('=@'): # 일부 예외적 데이터 수정
        tweet = tweet[1:]

    elif tweet.startswith('rt @'):
        tweet = tweet[3:]

    else:
        tweet = tweet 
        
    data_split = tokenizer.tokenize(tweet) # 단어 토큰화 

    no_contractions = [expand_contractions(x) for x in data_split] # 축약어 

    data_split2 = []

    for s in range(len(no_contractions)):
        partial = tokenizer.tokenize(no_contractions[s])

        for x in range(len(partial)):

            data_split2.append(partial[x])



    # @, #, http 로 시작하는 토큰 제거 
    trim_data = [x for x in data_split2 if not x.startswith('@') | x.startswith('=@')| x.startswith('http') | x.startswith('#')]
    
    # 이모티콘과 이모지 토큰 제거 
    trim_data = [w for w in trim_data if not w in emoticons]

    trim_data2 = []

    for s in range(len(trim_data)):
        #print(data_split[s])
        trim_data2.append(emoji_pattern.sub(r'', trim_data[s]))

    
    trim_data3 = []

    for s in range(len(trim_data2)):
        if len(trim_data2[s]) == 0:
            pass
        else:
            trim_data3.append(trim_data2[s])

    sample = ' '.join(trim_data3).strip()
    sample = tokenizer.tokenize(sample)

    # 영문자 이외 문자는 공백으로 변환
    only_english = list(filter(lambda t: t.isalpha(), sample))
    
    listToStr = ' '.join([str(elem) for elem in only_english]) 
    
    #무의미하게 반복된 철자 수정
    sentence = ''.join(['' if i > 1 and e == listToStr[i-2] and e == listToStr[i-1] else e for i,e in enumerate(listToStr)])

    sentence = re.sub(r'\s+', ' ', sentence)
    
    
    #표제어 추출
    
    sentence_lemma = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
    
    # 불용어 제거 

    tokens_without_stopwords = [x for x in sentence_lemma if x not in stopword_list]
    
    if len(tokens_without_stopwords) == 0:
        tokens_without_stopwords = 'NC'
    else:
        pass
        

    return tokens_without_stopwords

In [20]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

In [21]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [22]:
from multiprocessing import Pool
 
def use_multiprocess(func, iter, workers):
    pool = Pool(processes=workers)
    result = pool.map(func, iter)
    pool.close()
    return result

In [23]:
data.iloc[:2]

,Sentiment,Date,Blank,SentimentText
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...


In [24]:
import time
 
if __name__ == '__main__':
    start_time = time.time()
    
    data['TrimText_lemma'] = use_multiprocess(data_text_cleaning_lemma, data['SentimentText'], 10)
    
    print('실행 시간 :', round((time.time() - start_time)/60, 2), 'min')



실행 시간 : 31.58 min


In [26]:
data.iloc[:3]

,Sentiment,Date,Blank,SentimentText,TrimText_lemma
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...,"[upset, not, update, facebook, texting, might,..."
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...,"[dive, many, time, ball, manage, save, rest, g..."
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]"


In [ ]:
# 상대적으로 긴 문장 제거를 위해서 트위터 데이터 한 개당 토큰수 살펴보기

In [25]:
def count(data):
    return len(data)

In [27]:
import time
 
if __name__ == '__main__':
    start_time = time.time()
    
    data['TrimText_lemma_len'] = use_multiprocess(count, data['TrimText_lemma'], 10)
    
    print('실행 시간 :', round((time.time() - start_time)/60, 2), 'min')

실행 시간 : 0.07 min


In [28]:
data.iloc[:3]

,Sentiment,Date,Blank,SentimentText,TrimText_lemma,TrimText_lemma_len
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...,"[upset, not, update, facebook, texting, might,...",12
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...,"[dive, many, time, ball, manage, save, rest, g...",9
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]",6


# 전처리 후 토큰 수 확인 -> 토큰이 26개 넘는 데이터 삭제 

In [29]:
data.TrimText_lemma_len.max()

116

In [30]:
data.TrimText_lemma_len.median()

6.0

In [35]:
len(data[data.TrimText_lemma_len >= 26])

503

In [36]:
data2 = data[data.TrimText_lemma_len < 26]

In [37]:
len(data2)

1599497

In [39]:
len(data2[data2.Sentiment == 0])

799850

In [40]:
len(data2[data2.Sentiment == 1])

799647

# BERT 학습을 위해 token으로 되어 있는 데이터 str 형태로 만들기 

In [41]:
data2.TrimText_lemma[0]

['upset',
 'not',
 'update',
 'facebook',
 'texting',
 'might',
 'cry',
 'result',
 'school',
 'today',
 'also',
 'blah']

In [47]:
def join_data(txt):
    txt = ' '.join(txt)
    return txt

In [48]:
import time
 
if __name__ == '__main__':
    start_time = time.time()
    
    data2['TrimText_lemma_join'] = use_multiprocess(join_data, data2['TrimText_lemma'], 10)
    
    print('실행 시간 :', round((time.time() - start_time)/60, 2), 'min')

실행 시간 : 0.08 min


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [53]:
data2 = data2.rename(columns={"TrimText_lemma_len_join": "TrimText_lemma_join"})

In [54]:
data2.iloc[:3]

,Sentiment,Date,Blank,SentimentText,TrimText_lemma,TrimText_lemma_len,TrimText_lemma_join
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...,"[upset, not, update, facebook, texting, might,...",12,upset not update facebook texting might cry re...
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...,"[dive, many, time, ball, manage, save, rest, g...",9,dive many time ball manage save rest go bound
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]",6,whole body feel itchy like fire


In [55]:
len(data2)

1599497

In [51]:
type(data2.TrimText_lemma_len_join[0])

str

# 학습 데이터 및 테스트 데이터 만들기 
## 학습 데이터 100만개 
## 테스트 데이터 40만개 

In [60]:
data2.TrimText_lemma_len.median()

6.0

In [61]:
data2.TrimText_lemma_len.max()

25

In [56]:
neg = data2[data2.Sentiment == 0]

In [57]:
pos = data2[data2.Sentiment == 1]

In [58]:
len(neg)

799850

In [59]:
len(pos)

799647

In [62]:
train = neg[:500000].append(pos[:500000])

In [63]:
test = neg[500000:700000].append(pos[500000:700000])

In [64]:
len(train)

1000000

In [65]:
len(test)

400000

In [66]:
train.to_pickle("train_data_200131.pkl")

In [67]:
test.to_pickle("test_data_200131.pkl")